In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

In [ ]:
import os

pwd = os.getcwd()
pwd

In [ ]:
from VIDataloader.VIDataloader import get_dataloader
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((512, 512)), transforms.ToTensor(),
])
image_dir = os.path.join(pwd, "train_images", "images")
mask_dir = os.path.join(pwd, "train_images", "masks")
batch_size = 2

dataloader = get_dataloader(image_dir, mask_dir, batch_size, transform)
type(dataloader), len(dataloader)

In [ ]:
class Object(object):
    pass
opt = Object()
opt.t_stride = 5
opt.prev_warp = False
opt.double_size = False
opt.loss_on_raw = False
opt.batch_norm = False
opt.search_range = 0
opt.no_cuda = False
opt.model = 'vinet_final'
opt.crop_size = 512
opt.pretrain_path = None
opt.no_lstm = True

In [ ]:
import torch
import torch.nn.functional as F
from math import exp

def ssim(img1, img2, window_size=11, window_sigma=1.5, data_range=1.0, size_average=True):
    window = create_window(window_size, window_sigma).to(img1.device)

    mu1 = F.conv2d(img1, window, padding=window_size//2, groups=img1.shape[1])
    mu2 = F.conv2d(img2, window, padding=window_size//2, groups=img1.shape[1])

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=window_size//2, groups=img1.shape[1]) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=window_size//2, groups=img1.shape[1]) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=window_size//2, groups=img1.shape[1]) - mu1_mu2

    c1 = (0.01 * data_range) ** 2
    c2 = (0.03 * data_range) ** 2

    ssim_map = ((2 * mu1_mu2 + c1) * (2 * sigma12 + c2)) / ((mu1_sq + mu2_sq + c1) * (sigma1_sq + sigma2_sq + c2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)


def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2 / (2*sigma**2)) for x in range(window_size)])
    return gauss / gauss.sum()


def create_window(window_size, sigma):
    _1D_window = gaussian(window_size, sigma).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.expand(3, 1, window_size, window_size).contiguous()
    return window

In [ ]:
import torch.nn as nn
import torch.optim as optim
from model import generate_model
from train import train

criterion_L1 = nn.L1Loss()

def get_optimizer(model, lr=0.001):
    return optim.Adam(model.parameters(), lr=lr)

model = generate_model(opt)

In [ ]:
train(10, dataloader, model, criterion_L1, ssim, get_optimizer, opt)

In [ ]:
torch.save(model.state_dict(), 'model_weights.pt')